## Function calling

This new feature proves beneficial in a wide array of situations. It can assist in designing chatbots capable of interacting with various APIs, facilitate task automation, and enable extraction of organized information from inputs expressed in natural language.

In [9]:
!pip install --upgrade langchain
!pip install python-dotenv
!pip install openai

Ensure you got the AT LEAST Version 0.0.200

In [10]:
import pkg_resources


def print_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        print(f"The version of the {package_name} library is {version}.")
    except pkg_resources.DistributionNotFound:
        print(f"The {package_name} library is not installed.")


print_version("langchain")

The version of the langchain library is 0.0.202.


In [11]:
from dotenv import load_dotenv
import os
import openai
import json

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")



First, you need to define a function. This one mimics a database query with a "fake" price.

In [12]:
def get_pizza_info(pizza_name: str):
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

In [13]:
functions = [
    {
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Salami",
                },
            },
            "required": ["pizza_name"],
        },
    }
]

In [14]:
def chat(query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": query}],
        functions=functions,
    )
    message = response["choices"][0]["message"]
    return message

In [15]:
chat("What is the capital of france?")

<OpenAIObject at 0x14663008530> JSON: {
  "content": "The capital of France is Paris.",
  "role": "assistant"
}

In [8]:
query = "How much does pizza salami cost?"
message = chat(query)
message

<OpenAIObject at 0x14663008d10> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"pizza_name\": \"Salami\"\n}",
    "name": "get_pizza_info"
  },
  "role": "assistant"
}

In [16]:
if message.get("function_call"):
    function_name = message["function_call"]["name"]
    pizza_name = json.loads(message["function_call"]["arguments"]).get("pizza_name")
    function_response = get_pizza_info(
        pizza_name=pizza_name
    )

    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": query},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

second_response

<OpenAIObject chat.completion id=chatcmpl-7SIBO9mr9AL0vypVLj1q6pxa1RAtz at 0x14663008410> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Pizza salami typically costs $10.99.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686977434,
  "id": "chatcmpl-7SIBO9mr9AL0vypVLj1q6pxa1RAtz",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 59,
    "total_tokens": 69
  }
}

### LangChain 

LangChain allows you to use functions too, but currently (15.06.2023) it is kind of hacky to do it. 


In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage

In [19]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613")
message = llm.predict_messages(
    [HumanMessage(content="What is the capital of france?")], functions=functions
)
message

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False)

In [20]:
query = "How much does Pizza Salami cost in the restaurant?"
message_pizza = llm.predict_messages([HumanMessage(content=query)], functions=functions)
message_pizza

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_pizza_info', 'arguments': '{\n  "pizza_name": "Salami"\n}'}}, example=False)

In [21]:
message.additional_kwargs

{}

In [22]:
message_pizza.additional_kwargs

{'function_call': {'name': 'get_pizza_info',
  'arguments': '{\n  "pizza_name": "Salami"\n}'}}

In [23]:
import json

pizza_name = json.loads(message_pizza.additional_kwargs["function_call"]["arguments"]).get("pizza_name")
pizza_name

'Salami'

In [24]:
pizza_api_response = get_pizza_info(pizza_name=pizza_name)
pizza_api_response

'{"name": "Salami", "price": "10.99"}'

In [25]:


second_response = llm.predict_messages(
    [
        HumanMessage(content=query),
        AIMessage(content=str(message_pizza.additional_kwargs)),
        ChatMessage(
            role="function",
            additional_kwargs={
                "name": message_pizza.additional_kwargs["function_call"]["name"]
            },
            content=pizza_api_response
        ),
    ],
    functions=functions,
)
second_response

AIMessage(content='The Pizza Salami costs $10.99 in the restaurant.', additional_kwargs={}, example=False)

### Use Tools

You can convert Tools to functions, both the Tools provided by langchain and also your own tools. The Tool you see is there to show you the interface of a tool, it actually does nothing really do anything useful ;-) 

In [26]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class StupidJokeTool(BaseTool):
    name = "StupidJokeTool"
    description = "Tool to explain jokes about chickens"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        return "It is funny, because AI..."

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("joke tool does not support async")

In [27]:
from langchain.tools import format_tool_to_openai_function, MoveFileTool


tools = [StupidJokeTool(), MoveFileTool()]
functions = [format_tool_to_openai_function(t) for t in tools]
functions

[{'name': 'StupidJokeTool',
  'description': 'Tool to explain jokes about chickens',
  'parameters': {'properties': {'__arg1': {'title': '__arg1',
     'type': 'string'}},
   'required': ['__arg1'],
   'type': 'object'}},
 {'name': 'move_file',
  'description': 'Move or rename a file from one location to another',
  'parameters': {'title': 'FileMoveInput',
   'description': 'Input for MoveFileTool.',
   'type': 'object',
   'properties': {'source_path': {'title': 'Source Path',
     'description': 'Path of the file to move',
     'type': 'string'},
    'destination_path': {'title': 'Destination Path',
     'description': 'New path for the moved file',
     'type': 'string'}},
   'required': ['source_path', 'destination_path']}}]

In [29]:
query = "Why does the chicken cross the road? To get to the other side"
output = llm.predict_messages([HumanMessage(content=query)], functions=functions)
output

AIMessage(content='', additional_kwargs={'function_call': {'name': 'StupidJokeTool', 'arguments': '{\n"__arg1": "To get to the other side"\n}'}}, example=False)

In [30]:
question = json.loads(output.additional_kwargs["function_call"]["arguments"]).get("__arg1")
tool_response = tools[0].run(question)
tool_response

'It is funny, because AI...'

In [31]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=query),
        AIMessage(content=str(output.additional_kwargs)),
        ChatMessage(
            role="function",
            additional_kwargs={
                "name": output.additional_kwargs["function_call"]["name"]
            },
            content="""
                {tool_response}
            """,
        ),
    ],
    functions=functions,
)
second_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'StupidJokeTool', 'arguments': '{\n  "__arg1": "To get to the other side"\n}'}}, example=False)

### OpenAI Functions Agent

In [32]:
from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [33]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [35]:
agent.run("What is the capital of france?")



> Entering new  chain...
The capital of France is Paris.

> Finished chain.


'The capital of France is Paris.'

In [36]:
agent.run("What is 100 devided by 25?")



> Entering new  chain...

Invoking: `Calculator` with `100 / 25`




> Entering new  chain...
100 / 25```text
100 / 25
```
...numexpr.evaluate("100 / 25")...

Answer: 4.0
> Finished chain.
Answer: 4.0100 divided by 25 is equal to 4.

> Finished chain.


'100 divided by 25 is equal to 4.'